In [4]:
# ═══════════════════════════════════════════════════════════════════
# PREDICTION NOTEBOOK - Load models and make predictions
# ═══════════════════════════════════════════════════════════════════

import pandas as pd
import numpy as np
from drfp import DrfpEncoder
from olefin_descriptors import calculate_descriptors_for_dataset
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer


In [6]:
print("="*70)
print("LOADING MODELS FROM TRAINING NOTEBOOK")
print("="*70)

# ─────────────────────────────────────────────────────────────────
# 1. LOAD STORED OBJECTS FROM TRAINING NOTEBOOK
# ─────────────────────────────────────────────────────────────────
%store -r final_model
%store -r final_model_ee
%store -r scaler
%store -r imputer
%store -r reactant_desc_filtered
%store -r categorical_cols
%store -r dropped_features  # The list of features that were dropped

print("✅ Models and objects loaded")

# ─────────────────────────────────────────────────────────────────
# 2. LOAD TEST DATA
# ─────────────────────────────────────────────────────────────────
print("\n" + "="*70)
print("LOADING TEST DATA")
print("="*70)

test_df = pd.read_csv('test_data.csv')
print(f"Test data shape: {test_df.shape}")

# ─────────────────────────────────────────────────────────────────
# 3. PREPARE FEATURES (SAME AS TRAINING!)
# ─────────────────────────────────────────────────────────────────
print("\n" + "="*70)
print("PREPARING FEATURES")
print("="*70)

# 3a. Generate DRFP
print("\n[1/5] Generating DRFP...")
reaction_smiles = test_df['Reactant SMILES'] + '>>' + test_df['Product SMILES']
drfp_array = np.array(DrfpEncoder.encode(reaction_smiles.tolist(), n_folded_length=2048))
print(f"✓ DRFP shape: {drfp_array.shape}")

# 3b. Calculate olefin descriptors
print("\n[2/5] Calculating olefin descriptors...")
olefin_desc = calculate_descriptors_for_dataset(
    test_df,
    reactant_col='Reactant SMILES',
    product_col='Product SMILES'
)
print(f"✓ Olefin descriptors shape: {olefin_desc.shape}")

# 3c. Impute missing values
print("\n[3/5] Handling missing values...")
if olefin_desc.isnull().sum().sum() > 0:
    olefin_desc_imputed = pd.DataFrame(
        imputer.transform(olefin_desc),
        columns=olefin_desc.columns
    )
    print(f"✓ Imputed missing values")
else:
    olefin_desc_imputed = olefin_desc
    print(f"✓ No missing values")

# 3d. Drop same features as training (NO utils.py needed!)
print("\n[4/5] Filtering features...")
olefin_desc_filtered_test = olefin_desc_imputed.drop(columns=dropped_features)
print(f"✓ Dropped {len(dropped_features)} correlated features")
print(f"✓ Filtered to {olefin_desc_filtered_test.shape[1]} descriptors")

# 3e. Scale descriptors
print("\n[5/5] Scaling descriptors...")
olefin_desc_scaled = scaler.transform(olefin_desc_filtered_test)
print(f"✓ Descriptors scaled")

# 3f. Get categorical features
categorical = test_df[categorical_cols].values
print(f"✓ Categorical features: {categorical.shape}")

# 3g. Combine all features
X_test = np.hstack([drfp_array, olefin_desc_scaled, categorical])

print("\n" + "="*70)
print(f"✅ FINAL FEATURE MATRIX: {X_test.shape}")
print("="*70)

# ─────────────────────────────────────────────────────────────────
# 4. MAKE PREDICTIONS
# ─────────────────────────────────────────────────────────────────
print("\n" + "="*70)
print("MAKING PREDICTIONS")
print("="*70)

# Yield predictions
print("\n[1/2] Predicting yield...")
yield_pred = final_model.predict(X_test)
test_df['Predicted_Yield (%)'] = yield_pred
print(f"✓ Yield predictions: min={yield_pred.min():.2f}%, max={yield_pred.max():.2f}%")

# EE predictions
print("\n[2/2] Predicting ee (ddG er)...")
ee_pred = final_model_ee.predict(X_test)
test_df['Predicted_ddG_er (kcal/mol)'] = ee_pred
print(f"✓ EE predictions: min={ee_pred.min():.3f}, max={ee_pred.max():.3f} kcal/mol")

# ─────────────────────────────────────────────────────────────────
# 5. DISPLAY AND SAVE RESULTS
# ─────────────────────────────────────────────────────────────────
print("\n" + "="*70)
print("RESULTS")
print("="*70)

# Display first few predictions
print("\n📊 First 10 predictions:")
display_cols = ['Reactant SMILES', 'Predicted_Yield (%)', 'Predicted_ddG_er (kcal/mol)']
print(test_df[display_cols].head(10))

# Save to CSV
output_file = 'predictions.csv'
test_df.to_csv(output_file, index=False)
print(f"\n✅ All predictions saved to '{output_file}'")

# Summary statistics
print("\n📈 Prediction Summary:")
print(f"  Yield:  Mean = {yield_pred.mean():.2f}%, Std = {yield_pred.std():.2f}%")
print(f"  EE:     Mean = {ee_pred.mean():.3f} kcal/mol, Std = {ee_pred.std():.3f} kcal/mol")

print("\n" + "="*70)
print("🎉 PREDICTIONS COMPLETE!")
print("="*70)

LOADING MODELS FROM TRAINING NOTEBOOK
no stored variable or alias dropped_features
no stored variable or alias #
no stored variable or alias The
no stored variable or alias list
no stored variable or alias of
no stored variable or alias features
no stored variable or alias that
no stored variable or alias were
no stored variable or alias dropped
✅ Models and objects loaded

LOADING TEST DATA


FileNotFoundError: [Errno 2] No such file or directory: 'test_data.csv'